
# CNEFE 2022 — Saída *Wide v3* (categorias 1..8 + subtipos para OUTRAS_FINALIDADES)

Este notebook reproduz o fluxo utilizado para o CNEFE 2022:

- Mapeia `COD_ESPECIE` (1..8) diretamente para colunas 0/1 (`cat_*`).
- Gera subtipos (`sub_*`) **apenas** quando `COD_ESPECIE == 6` (OUTRAS_FINALIDADES), via palavras‑chave em `DSC_ESTABELECIMENTO`.
- Exporta:
  - `outputs/cnefe2022_wide_v3.gpkg` (camada `cnefe2022_wide_v3`)
  - `outputs/cnefe2022_wide_v3.geojson`
  - `outputs/cnefe2022_aggr_face_v3.csv` (agregado por `NUM_FACE`)
  - `outputs/cnefe2022_aggr_setor_v3.csv` (agregado por `COD_SETOR`)

> **Ajustes usuais**: alterar `INPUT_JSON` e editar o dicionário `SUBTIPO_KEYWORDS`.


In [ ]:

# (Opcional) Dependências se estiver em um ambiente "limpo"
# !pip -q install geopandas pyogrio shapely fiona pyproj


In [1]:

from pathlib import Path
import pandas as pd
import geopandas as gpd

# Caminhos de entrada/saída
INPUT_JSON = Path('C:/Users/Ana Maffini/Downloads/qg_810_endereco_Munic4310009.json/qg_810_endereco_Munic4310009.json')  # ajuste conforme necessário
OUT_DIR    = Path('C:/Users/Ana Maffini/Downloads/qg_810_endereco_Munic4310009.json/outputs'); OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_GPKG    = OUT_DIR / 'cnefe2022_wide_v3.gpkg'
OUT_GEOJSON = OUT_DIR / 'cnefe2022_wide_v3.geojson'
OUT_FACE    = OUT_DIR / 'cnefe2022_aggr_face_v3.csv'
OUT_SETOR   = OUT_DIR / 'cnefe2022_aggr_setor_v3.csv'

# Leitura
gdf = gpd.read_file(INPUT_JSON)
print('CRS:', gdf.crs, '| Registros:', len(gdf))
print('Campos:', list(gdf.columns)[:20], '...')


CRS: EPSG:4674 | Registros: 12141
Campos: ['COD_UNICO_ENDERECO', 'COD_UF', 'COD_MUNICIPIO', 'COD_SETOR', 'NUM_QUADRA', 'NUM_FACE', 'CEP', 'DSC_LOCALIDADE', 'LOGRAD_NUM', 'COMPLEMENTO', 'NV_GEO_COORD', 'COD_TIPO_ESPECI', 'DSC_ESTABELECIMENTO', 'COD_ESPECIE', 'geometry'] ...


In [2]:

# Normalização
def as_str(x):
    return None if pd.isna(x) else str(x).strip()

for col in ['COD_UNICO_ENDERECO','COD_UF','COD_MUNICIPIO','COD_SETOR','CEP',
            'DSC_LOCALIDADE','LOGRAD_NUM','COMPLEMENTO','COD_ESPECIE','DSC_ESTABELECIMENTO']:
    if col in gdf.columns:
        gdf[col] = gdf[col].apply(as_str)

def zpad(x, n):
    if x is None:
        return None
    return x.zfill(n) if x.isdigit() else x

if 'COD_UF' in gdf.columns:        gdf['COD_UF'] = gdf['COD_UF'].apply(lambda v: zpad(v, 2) if v else v)
if 'COD_MUNICIPIO' in gdf.columns: gdf['COD_MUNICIPIO'] = gdf['COD_MUNICIPIO'].apply(lambda v: zpad(v, 7) if v else v)
if 'CEP' in gdf.columns:           gdf['CEP'] = gdf['CEP'].apply(lambda v: zpad(v, 8) if v else v)

gdf['NUM_FACE']  = gdf.get('NUM_FACE')
gdf['COD_SETOR'] = gdf.get('COD_SETOR')


In [3]:

# Mapeamento direto (CNEFE 2022)
code_to_cat = {
    '1': 'DOMICILIO_PARTICULAR',
    '2': 'DOMICILIO_COLETIVO',
    '3': 'AGROPECUARIO',
    '4': 'ENSINO',
    '5': 'SAUDE',
    '6': 'OUTRAS_FINALIDADES',
    '7': 'CONSTRUCAO_REFORMA',
    '8': 'RELIGIOSO',
}

esp = gdf['COD_ESPECIE'].fillna('').astype(str).str.strip()

# Cria colunas cat_*
for code, name in code_to_cat.items():
    gdf[f'cat_{name}'] = (esp == code).astype('int64')

# (opcional) flag de não classificado (casos fora de 1..8)
valid = set(code_to_cat.keys())
gdf['cat_NAO_CLASSIFICADO'] = (~esp.isin(valid)).astype('int64')

gdf.filter(like='cat_').sum().sort_index()


cat_AGROPECUARIO            1241
cat_CONSTRUCAO_REFORMA       212
cat_DOMICILIO_COLETIVO         7
cat_DOMICILIO_PARTICULAR    9221
cat_ENSINO                    28
cat_NAO_CLASSIFICADO           0
cat_OUTRAS_FINALIDADES      1326
cat_RELIGIOSO                 51
cat_SAUDE                     55
dtype: int64

In [ ]:

# Subtipos apenas para OUTRAS_FINALIDADES (6)
import re

def norm_text(x):
    if not x or pd.isna(x):
        return ''
    x = str(x).upper()
    tr = str.maketrans('ÁÂÃÀÄÉÊËÍÎÏÓÔÕÖÚÜÇ', 'AAAAAEEEIIIOOOOUUC')
    return x.translate(tr)

txt = gdf['DSC_ESTABELECIMENTO'].fillna('').map(norm_text)
mask_outros = gdf['cat_OUTRAS_FINALIDADES'] == 1

SUBTIPO_KEYWORDS = {
    'MERCADO': ['MERCAD', 'MINIMERCADO', 'MINI MERCADO', 'MERCADINHO', 'MERCEARIA', 'MERCADO', 'QUITANDA', 'FRUTEIRA', 'EMPORIO', 'EMPÓRIO'],
    'SUPERMERCADO' : ['SUPERMERCADO', 'HIPERMERCADO'],
    'ACOUGUE': ['ACOUGUE', 'AÇOUGUE', 'AÇOGUE', 'ACOGUE'],
    'LAZER': ['AUDITORIO', 'ARENA', 'SHOW', 'TEATRO', 'CASA DE FESTA'],
    'PADARIA': ['PADARIA', 'PANIFIC', 'CONFEITARIA', 'PANIFICADORA'],
    'CAFETERIA': ['CAFETERIA', 'CAFE', 'CAFÉ'],
    'FARMACIA':['FARMACIA', 'DROGARIA', 'PANVEL', 'SAO JOAO', 'SÃO JOÃO', 'DROGARAIA', 'DROGA RAIA', 'RAIA', 'FARMÁCIA'],
    'BANCO': ['BANCO', 'BRADESCO', 'ITAU', 'SANTANDER', 'BANRISUL', 'SICREDI', 'SICOOB', 'CAIXA', 'CEF', 'FINANCEIRA', 'CONSORCIO', 'CONSÓRCIO'],
    'BAR/BOATE': ['BAR', 'BOTECO', 'PUB', 'BARR', 'BOATE', 'CLUB'],
    'RESTAURANTE': ['RESTAURANT', 'PIZZARIA', 'CHURRASC'],
    'LANCHERIA': ['LANCHERIA', 'LANCHONETE', 'LANCH'],
    'COMERCIO': ['LOJA', 'COMERC', 'ARMAZEM', 'ARMAZÉM', 'BAZAR', 'VAREJO', 'ATACAD', 'COMERCIO', 'COMÉRCIO', 'ATACADO', 'VAREJISTA', 'LIVRARIA', 'RENNER', 'AMERICANAS'],
    'SERVICOS': ['OFICINA', 'LAVACAO', 'SALAO', 'CARTORIO', 'IMOBILIARIA', 'IMOBILIÁRIA', 'CONTABILIDADE', 'ESCRITORIO', 'LAN HOUSE', 'LANHOUSE'],
    'SHOPPING': [ 'SHOPPING', 'GALERIA COMERCIAL', 'IGUATEMI', 'GALERIA DE LOJAS', 'CENTRO COMERCIAL'],
    'TELECOMUNICACAO': ['TELECOM', 'TELECOMUNICACAO', 'TELECOMUNICAÇÃO', 'CENTRAL TELEFONICA', 'CENTRAL TELEFÔNICA', 'TELEFONIA', 'TELEFONICA', 'TELEFÔNICA', 'EMBRATEL'],
    'HOSPEDAGEM': ['HOTEL', 'POUSADA', 'MOTEL', 'HOSTEL', 'PENSAO', 'PENSOES', 'PENSÃO', 'PENSÕES'],
    'INSTIT RESIDENCIAL': ['ASILO', 'CASA DE REPOUSO', 'REPOUSO', 'MORADIA ESTUDANTIL', 'CASA DE ESTUDANTE', 'CASA DE ESTUDANTES'],
    'CULTURA/LAZER': ['CLUBE', 'CINEMA', 'TEATRO', 'BOLICHE'],
    'CRECHES': ['CRECHE', 'BERCARIO', 'BERÇÁRIO', 'EDUCAÇÃO INFANTIL', 'CENTRO DE EDUCAÇÃO INFANTIL', 'JARDIM DE INFÂNCIA', 'JARDIM', 'CENTRO EDUCAÇÃO INFANTIL'],
    'ENSINO': ['ESCOLA', 'ENSINO', 'COLEGIO', 'COLÉGIO', 'CENTRO EDUCACIONAL'],
    'OUTRA EDUCACAO': ['AUTOESCOLA', 'AUTO ESCOLA', 'CURSINHO', 'PREPARATORIO', 'PREPARATÓRIO', 'CCAA', 'CURSO', 'WIZARD', 'ESCOLA PROFISSIONAL', 'ESCOLA DE MUSICA', 'ESCOLA MUSICA'],
    'UNIVERSIDADE': ['UNIVERSIDADE', 'SUPERIOR', 'ULBRA', 'UFN', 'UNISSINOS', 'UNISINOS', 'UNIJUI', 'FACULDADE', 'UFRGS', 'UFPEL', 'IF', 'INSTITUTO FEDERAL'],
    'ESTAB ESPORTIVOS':['CLUBE', 'GINASIO', 'GINÁSIO', 'CAMPO DE FUTEBOL', 'ESCOLINHA DE FUTEBOL', 'CENTRO ESPORTIVO', 'HARAS,' 'HIPICA', 'HÍPICA', 'QUADRA', 'FUTEBOL', 'CAMPO DE GOLFE'],
    'INDUSTRIA': ['FABRICA', 'INDUSTR', 'USINA'],
    'CORRREIOS': ['CORREIO', 'CORREIOS'],
    'BIBLIOTECA':['BIBLIOTECA', 'BIBLIO'],
    'MUSEUS/GALERIAS': ['MUSEU', 'GALERIA', 'GALERIA DE ARTE'],
    'CENTRO COMUNITARIO': ['COMUNITARIO', 'COMUNITÁRIO', 'CENTRO COMUNITARIO', 'CENTRO COMUNITÁRIO', 'MORADORES', 'DEFICIENTES', 'ASSOCIACAO', 'ASSOCIAÇÃO', 'CENTRO SOCIAL', 'SOCIEDADE', 'SERVIDORES', 'FUNDACAO', 'FUNDAÇÃO', 'CONSELHO', 'CENTRO DE CONVIVENCIA', 'CENTRO DE CONVIVÊNCIA', 'CENTRO DE ATENCAO', 'CENTRO DE ATENÇÃO'],
    'HOSPITAL': ['HOSPITAL', 'EMERGENCIA'],
    'SAUDE': ['CLINICA', 'UNIMED', 'SAUDE', 'CONSULTORIO', 'CONSULTÓRIO', 'POLICLINICA','POLICLÍNICA', 'DENTISTA', 'ODONTOLOGIA', 'ACUMPUNTURA', 'CARDIOLOGIA', 'CENTRO DE SAUDE', 'CENTRO DE SAÚDE', 'CENTRO MEDICO', 'CENTRO MÉDICO', 'CENTRO ODONTOLOGICO', 'CENTRO ODONTOLÓGICO', 'ODONTO', 'MASSOTERAPIA', 'MEDICINA', 'PSICOLOGIA', 'PSICOLOGO', 'PSICÓLOGO', 'RADIOLOGIA', 'PROTESE', 'PRÓTESE', 'TERAPEUTICO', 'TERAPÊUTICO', 'ORTODONTIA', 'ORTODONTISTA', 'ORTOPEDIA', 'PSICOLOGA', 'PSICÓLOGA', 'PSICOPEDAGOGA', 'PSICOTERAPEUTA', 'PSIQUIATRA', 'CENTRO DE RECUPERACAO', 'FISIOTERAPIA'],
    'VETERINARIA': ['VETERINARIA', 'VETERINARIO', 'VETERINÁRIA', 'VETERINÁRIO'],
    'ESTAB RELIGIOSO': ['IGREJA', 'CULTO', 'TEMPLO', 'ASSEMBLEIA DE DEUS', 'ASSEMBLEIA DEUS', 'ESPIRITA', 'UMBANDA', 'CAPELA', 'PAROQUIA', 'RELIGIOSO', 'RELIGIOSA', 'DIOCESE', 'CONGREGACAO', 'CASA PAROQUIAL', 'IGREGA'], 
    'POSTO DE SAUDE': ['POSTO DE SAUDE', 'UBS', 'CRAS', 'CAPES'],
    'AREAS VERDES': ['PRACA', 'PRAÇA', 'PARQUE', 'PRACINHA', 'VERDE'],
    'ACADEMIA': ['ACADEMIA', 'YOGA', 'IOGA', 'PILATES', 'CROSSFIT', 'GINASTICA', 'GINÁSTICA'],
    'ESTACIONAMENTO':['GARAGEM', 'ESTACIONAMENTO', 'ESTACIONAMENTOS', 'ESTACIONAMENTP'],
    'GARAGENS':['GARAGEM DE AUTOMÓVEIS', 'GARAGEM DE AUTOMOVEIS', 'LOCADORA DE VEÍCULOS', 'AUTOLOCADORA', 'REVENDEDORA', 'REVENDEDORA DE CARROS', 'REVENDEDORA DE VEÍCULOS', 'REVENDEDORA DE MOTOS', 'CONCESSIONÁRIA', 'REVENDA'],
    'TRANSPORTE': ['RODOVIARIA', 'RODOVIÁRIA', 'ESTACAO', 'ESTAÇÃO' , 'TERMINAL', 'AEROPORTO', 'AERPORTO'],
    'SEGURANCA': ['DELEGACIA', 'POLICIA', 'POLÍCIA', 'BOMBEIRO', 'BRIGADA', 'BATALHÃO', 'BATALHAO', 'GUARDA MUNICIPAL', 'QUARTEL', 'PLICIA', 'EXÉRCITO', 'AERONAUTICA', 'AERONÁUTICA'],
    'ADMIN_PUBLICA': ['PREFEIT', 'PREFEITURA','FÓRUM', 'FORUM', 'CAMARA', 'CÂMARA', 'CREAS', 'INSS', 'SECRETARIA', 'MINISTERIO', 'TRIBUNAL', 'GABINETE', 'SINDICATO', 'DEPARTAMENTO', 'PROCURADORIA', 'CONSELHO TUTELAR', 'PROCON', 'DNIT', 'CENTRO LEGISLATIVO', 'LEGISLATIVO', 'EXECUTIVO', 'GABINETE'],
    'CEMITERIO/CREMATORIO': ['CEMITERIO', 'CEMITÉRIO', 'CREMATORIO', 'CREMATÓRIO'],

}

# Inicializa sub_* com zeros
for key in SUBTIPO_KEYWORDS.keys():
    gdf[f'sub_{key}'] = 0

# Aplica apenas onde OUTRAS_FINALIDADES == 1
txt_outros = txt.where(mask_outros, '')
for key, pats in SUBTIPO_KEYWORDS.items():
    pat = re.compile('|'.join(map(re.escape, pats)))
    gdf.loc[txt_outros.str.contains(pat), f'sub_{key}'] = 1

# Fallback: OUTRAS sem subtipo reconhecido
gdf['sub_OUTROS'] = (mask_outros & (gdf.filter(like='sub_').sum(axis=1) == 0)).astype('int64')

gdf.filter(like='sub_').sum().sort_index()


sub_ADMIN_PUBLICA      9
sub_BANCO             10
sub_BAR               47
sub_COMERCIO         106
sub_CULTURA_LAZER     24
sub_HOSPEDAGEM         6
sub_INDUSTRIA         27
sub_MERCADO           37
sub_OUTROS           885
sub_PADARIA            6
sub_RESTAURANTE       26
sub_SEGURANCA          3
sub_SERVICOS         143
sub_TRANSPORTE        15
dtype: int64

In [5]:

# Exportações
if gdf.crs is None:
    gdf = gdf.set_crs(4674)  # SIRGAS 2000

# GeoPackage (mantém CRS original)
gdf.to_file(OUT_GPKG, layer='cnefe2022_wide_v3', driver='GPKG')

# GeoJSON (em WGS84) — reprojecta se necessário
gdf_wgs84 = gdf.to_crs(4326) if (gdf.crs and gdf.crs.to_epsg() != 4326) else gdf
gdf_wgs84.to_file(OUT_GEOJSON, driver='GeoJSON')

print('GPKG:', OUT_GPKG)
print('GeoJSON:', OUT_GEOJSON)


GPKG: C:\Users\Ana Maffini\Downloads\qg_810_endereco_Munic4310009.json\outputs\cnefe2022_wide_v3.gpkg
GeoJSON: C:\Users\Ana Maffini\Downloads\qg_810_endereco_Munic4310009.json\outputs\cnefe2022_wide_v3.geojson


In [6]:

# Agregações estilo 2010 — somatório das colunas cat_ e sub_
dum_cols = [c for c in gdf.columns if c.startswith('cat_') or c.startswith('sub_')]

face_aggr  = gdf.groupby('NUM_FACE',  dropna=False)[dum_cols].sum().reset_index()
setor_aggr = gdf.groupby('COD_SETOR', dropna=False)[dum_cols].sum().reset_index()

face_aggr.to_csv(OUT_FACE,  index=False, encoding='utf-8')
setor_aggr.to_csv(OUT_SETOR, index=False, encoding='utf-8')

print('Agregado FACE:', OUT_FACE)
print('Agregado SETOR:', OUT_SETOR)


Agregado FACE: C:\Users\Ana Maffini\Downloads\qg_810_endereco_Munic4310009.json\outputs\cnefe2022_aggr_face_v3.csv
Agregado SETOR: C:\Users\Ana Maffini\Downloads\qg_810_endereco_Munic4310009.json\outputs\cnefe2022_aggr_setor_v3.csv


In [7]:

# Resumo rápido para conferência
summary = (gdf.filter(like='cat_').sum().to_frame('total_1s')
           .merge(gdf.filter(like='sub_').sum().to_frame('total_1s'), 
                  left_index=True, right_index=True, how='outer')
           .fillna(0).astype(int))
summary.sort_index()


,total_1s_x,total_1s_y
cat_AGROPECUARIO,1241,0
cat_CONSTRUCAO_REFORMA,212,0
cat_DOMICILIO_COLETIVO,7,0
cat_DOMICILIO_PARTICULAR,9221,0
cat_ENSINO,28,0
cat_NAO_CLASSIFICADO,0,0
cat_OUTRAS_FINALIDADES,1326,0
cat_RELIGIOSO,51,0
cat_SAUDE,55,0
sub_ADMIN_PUBLICA,0,9
